In [ ]:
#!pip install dice-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.3 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you 

In [ ]:
#!pip install --upgrade numpy
#!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
mizani 0.13.3 requires pandas>=2.2.0, but you have

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from dice_ml import Data, Model, Dice

df = pd.read_csv("student_depression_dataset.csv")
df.replace({'Yes': 1, 'No': 0}, inplace=True)
meta_columns = ['id', 'Gender', 'Age', 'City', 'Profession', 'Depression']
df_meta = df[meta_columns].copy()
y = df['Depression'].astype(int)
X = df.drop(columns=['Depression'])


<ipython-input-1-47edca961c75>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Yes': 1, 'No': 0}, inplace=True)


In [ ]:

X = pd.get_dummies(X)
X = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X), columns=X.columns)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)


In [ ]:
print([col for col in X.columns if "suicidal" in col.lower()])


['Have you ever had suicidal thoughts ?']


In [ ]:

top_cf_features = [
    'Have you ever had suicidal thoughts ?',
    'Academic Pressure',
    'Financial Stress_5.0',
    'Financial Stress_1.0',
    'Work/Study Hours',
    'Study Satisfaction',
    'Financial Stress_2.0',
    'Dietary Habits_Unhealthy',
    'Dietary Habits_Healthy'
]

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_top = pd.DataFrame(X_train, columns=X_scaled.columns)[top_cf_features]
X_test_top = pd.DataFrame(X_test, columns=X_scaled.columns)[top_cf_features]
_, df_meta_test = train_test_split(df_meta, test_size=0.2, stratify=y, random_state=42)


In [ ]:

lr_model_top = LogisticRegression(solver='liblinear', random_state=42)
lr_model_top.fit(X_train_top, y_train)


LogisticRegression(random_state=42, solver='liblinear')

In [ ]:

dice_data = Data(
    dataframe=pd.concat([X_train_top, pd.Series(y_train, name='Depression')], axis=1),
    continuous_features=top_cf_features,
    outcome_name='Depression'
)
dice_model = Model(model=lr_model_top, backend='sklearn', model_type='classifier')
exp = Dice(dice_data, dice_model, method='genetic')


In [ ]:

y_pred = lr_model_top.predict(X_test_top)
depressed_indices = np.where(y_pred == 1)[0]
selected_indices = np.random.choice(depressed_indices, size=50, replace=False)
X_depressed_50 = X_test_top.iloc[selected_indices].reset_index(drop=True)
meta_selected = df_meta_test.iloc[selected_indices].reset_index(drop=True)


In [ ]:

cf_combined_all = []
for i in range(len(X_depressed_50)):
    instance = X_depressed_50.iloc[[i]]
    try:
        dice_exp = exp.generate_counterfactuals(instance, total_CFs=1, desired_class="opposite")
        df_cf = dice_exp.cf_examples_list[0].final_cfs_df
        if df_cf is not None and not df_cf.empty:
            changed = [feat for feat in instance.columns if instance.iloc[0][feat] != df_cf.iloc[0][feat]]
            original = instance.reset_index(drop=True).add_prefix("original_")
            counterfactual = df_cf.reset_index(drop=True).add_prefix("cf_")
            metadata = meta_selected.iloc[[i]].reset_index(drop=True)
            combined = pd.concat([metadata, original, counterfactual], axis=1)
            combined["changed_features"] = ", ".join(changed)
            cf_combined_all.append(combined)
        else:
            print(f"No CF for index {i}")
    except Exception as e:
        print(f"Error at index {i}: {e}")


100%|██████████| 1/1 [00:00<00:00,  5.68it/s]


In [ ]:

if cf_combined_all:
    cf_combined_df = pd.concat(cf_combined_all).reset_index(drop=True)
    cf_combined_df.to_csv("counterfactuals_logreg_top50_with_metadata.csv", index=False)
    print("✅ Saved: counterfactuals_logreg_top50_with_metadata.csv")
else:
    print("⚠️ No counterfactuals generated.")


✅ Saved: counterfactuals_logreg_top50_with_metadata.csv


In [ ]:
avg_instance = pd.DataFrame([X_depressed_50.mean()])

# Show the input instance before generating CFs
print("🟦 Original (Average Depressed Student):")
display(avg_instance)

# Check if the model predicts them as depressed
if lr_model_top.predict(avg_instance)[0] == 1:
    try:
        cf_avg = exp.generate_counterfactuals(avg_instance, total_CFs=3, desired_class="opposite")
        cf_avg_df = cf_avg.cf_examples_list[0].final_cfs_df

        if cf_avg_df is not None and not cf_avg_df.empty:
            cf_avg_df.to_csv("counterfactuals_logreg_avg_student.csv", index=False)
            print("\n✅ Counterfactuals Generated:")
            display(cf_avg_df)  # Show counterfactuals (after)
        else:
            print("❌ No counterfactuals generated for average student.")
    except Exception as e:
        print(f"⚠️ Error generating counterfactuals for average student: {e}")
else:
    print("ℹ️ The average student is not predicted as depressed — no CF needed.")


🟦 Original (Average Depressed Student):


,Have you ever had suicidal thoughts ?,Academic Pressure,Financial Stress_5.0,Financial Stress_1.0,Work/Study Hours,Study Satisfaction,Financial Stress_2.0,Dietary Habits_Unhealthy,Dietary Habits_Healthy
0,0.595784,0.534794,0.138781,-0.319139,0.319081,-0.149757,0.203997,0.145478,-0.345691


100%|██████████| 1/1 [00:00<00:00,  4.11it/s]


✅ Counterfactuals Generated:


,Have you ever had suicidal thoughts ?,Academic Pressure,Financial Stress_5.0,Financial Stress_1.0,Work/Study Hours,Study Satisfaction,Financial Stress_2.0,Dietary Habits_Unhealthy,Dietary Habits_Healthy,Depression
0,-0.916288,-0.231247,0.438651,-0.190995,0.507966,-0.019417,0.601700,-0.618778,0.390215,0
0,-1.312775,0.621660,-0.562987,-0.474133,0.227377,0.041262,-0.470728,-0.765981,-0.614676,0
0,0.384074,-0.478963,0.404393,0.968571,0.900219,1.332426,1.086180,0.200799,-0.018541,0
